# Combined Cost Analysis

We is the fun part. Now I will know the best time for my ski trip. I'm already excited. 

I just need to combine data for a 4 night stay ski trip for possible travel start date between 15th December 2018 and 7th April 2019. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.DataFrame(data={'date':pd.date_range(start='2018-12-15', end='2019-04-21')})
df.info()
# df = df.set_index('date')
# df['date'].head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 1 columns):
date    128 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.1 KB


## Airbnb accommodation data

### Narrow down listing according to preferences
First, I narrow down the listing according to following criteria:
1. Number of people a listing accommodates: 2-4 (This would depend on how many people I am able to convince to join me on the trip).
2. Minimum listing rating of: 80
3. Room type: Entire home/apt

In [2]:
listing = pd.read_csv('../../Data/VacationPlanner/advanced_listing.csv')
idx = ((listing['accommodates'] >= 2) &
       (listing['accommodates'] <= 4) &
       (listing['review_scores_rating'] >= 90) &
       (listing['room_type'] >= 'Entire home/apt'))

selected_listing_id = listing[idx]['id']
listing = listing.set_index('id')

### Narrow down calendar price data to selected listing id

In [3]:
calendar = pd.read_csv('../../Data/VacationPlanner/advanced_calendar.csv')
calendar = calendar[calendar['id'].isin(selected_listing_id)]
calendar.head()
calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121404 entries, 151 to 262739
Data columns (total 4 columns):
Unnamed: 0    121404 non-null int64
id            121404 non-null int64
date          121404 non-null object
price         111671 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 4.6+ MB


### Compile total median Airbnb cost 

For each start date between 15th December 2018 and 7th April 2019, compile the total median cost of all the listing in which I could potentially stay for 4 consecutive nights


In [4]:
def median_airbnb_cost(start_date):
    
    # Formating for start and end date
    start_date = str(start_date)[:10]
    df_ = pd.DataFrame(data={'date':pd.date_range(start=start_date, periods=4)})
    end_date = df_['date'].astype(str).tolist()[-1]
    
    # Collect all listing between start and end date
    sub_calendar = calendar[(calendar['date'] >= start_date) &
                           (calendar['date'] <= end_date)]
    
    # Find listings which are not available for all the 4 nights
    invalid_id = sub_calendar[sub_calendar['price'].isnull()]['id'].unique()
    
    # Sum of price to stay at a listing
    df_local = sub_calendar.groupby('id').sum()
    
    # Remove invalid id
    df_local = df_local[~df_local.index.isin(invalid_id)]
    
    # Compure price person for the valid listings
    df_local['id'] = df_local.index
    df_local.reset_index(drop=True)
    df_local['cleaning_fee'] = df_local['id'].apply(lambda x: listing.loc[x, 'cleaning_fee'])
    df_local['accommodates'] = df_local['id'].apply(lambda x: listing.loc[x, 'accommodates'])
    df_local['price'] = (df_local['price'] + df_local['cleaning_fee'])/df_local['accommodates']
    
    return df_local['price'].median()

df['Airbnb_cost'] = df['date'].apply(median_airbnb_cost)

In [5]:
# plt.plot(df['date'], df['Airbnb cost'])
df.head()

,date,Airbnb_cost
0,2018-12-15,148.333333
1,2018-12-16,147.250000
2,2018-12-17,147.000000
3,2018-12-18,147.500000
4,2018-12-19,150.000000


## Flight price data

To df add the flight price for the given start date with 4 nights of stay

In [10]:
flight_price = pd.read_csv('../../Data/VacationPlanner/flight_price.csv')

# df['Madison'] = df['date'].apply(lambda x: flight_price.loc[str(x)[:10],'Madison'])

df.head()

,date,Airbnb_cost
0,2018-12-15,148.333333
1,2018-12-16,147.250000
2,2018-12-17,147.000000
3,2018-12-18,147.500000
4,2018-12-19,150.000000
